use simplified data 1

data source: ./simplified_data/simplified_1.csv

data v2 取用藥1-5 款, 刪除全0 col, 處方&針炙ignored, 文字input ignored

deleted 回診,西藥

In [133]:
import numpy as np
import pandas as pd
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy



In [134]:
def ReadData(FILENAME):
    data = pd.read_csv(FILENAME)
    print("in ReadData")
    print("type of data:", type(data))
    print(f'Shape of data = ({data.shape[0]} rows, {data.shape[1]} cols).')
    return data

def SplitXY(data, data_left_bound, label_left_bound, data_right_bound=None, label_right_bound=None):
    """label_left_bound: 第一個藥材的col no."""
    # Body status: 1~3, Diagnosis: 4~7, Symptom: 11~124
    # Prescription: 125~226
    # split_X = list(range(0, label_left_bound))
    # split_Y = list(range(label_left_bound, len(data.columns)))
    if label_right_bound is None:
        label_right_bound = len(data.columns)
    if data_right_bound is None:
        data_right_bound = label_left_bound
    X = data.iloc[1:, list(range(data_left_bound, data_right_bound))]
    y = data.iloc[1:, list(range(label_left_bound, label_right_bound))]
    
    # Debug
    print("SplitXY:")
    print(f'Shape of X = ({X.shape[0]} rows, {X.shape[1]} cols). First 10 data of X:')
    print(X.iloc[:10, :10])
    print(f'Shape of y = ({y.shape[0]} rows, {y.shape[1]} cols). First 10 data of y:')
    print(y.iloc[:10, :10])
    return X, y

In [135]:
data_pd = ReadData("./simplified_data/simplified_1.csv")

in ReadData
type of data: <class 'pandas.core.frame.DataFrame'>
Shape of data = (113 rows, 180 cols).


In [136]:
first_medicine_idx = 100
chosen_medicine='甘草'
chosen_medicine_idx = None
print("first_medicine: ",data_pd.columns[first_medicine_idx])
print("should be 麻黃")


for i in range(len(data_pd.columns.tolist())):
    if data_pd.columns.tolist()[i] == chosen_medicine:
        chosen_medicine_idx = i
        break
print("chosen_medicine_idx: ", chosen_medicine_idx)
print("chosen_medicine: " ,data_pd.columns.tolist()[chosen_medicine_idx])


first_medicine:  麻黃
should be 麻黃
chosen_medicine_idx:  167
chosen_medicine:  甘草


In [137]:
# now try only by 1 mdeicine

X,y = SplitXY(data_pd, data_left_bound=2, data_right_bound=first_medicine_idx, 
              label_left_bound=chosen_medicine_idx, label_right_bound=chosen_medicine_idx+1)
X_np = X.values.astype('float64')
y_np = y.values.astype('float64')


SplitXY:
Shape of X = (112 rows, 98 cols). First 10 data of X:
    乳癌  肺癌  肝癌  腺癌  攝護腺癌  骨癌  腦瘤  肝炎  水腫  二便
1    0   0   0   0     0   0   0   0   0   0
2    0   0   0   0     0   0   0   0   0   0
3    0   0   0   0     0   0   0   0   1   0
4    0   0   0   0     0   0   0   0   0   1
5    0   0   0   0     0   0   0   0   0   0
6    0   0   0   0     0   0   0   0   0   0
7    0   0   0   0     0   0   0   0   0   0
8    0   0   0   0     0   0   0   0   0   0
9    0   0   0   0     0   0   0   1   0   0
10   0   0   0   0     0   0   1   0   0   0
Shape of y = (112 rows, 1 cols). First 10 data of y:
    甘草
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
10   0


In [138]:
print("shape of X_np", X_np.shape)
print("shape of y_np", y_np.shape)
input_dim = X_np.shape[0]

shape of X_np (112, 98)
shape of y_np (112, 1)


In [139]:
model = Sequential([
    Dense (units=16, input_shape=(98,), activation='relu'),
    Dense (units=32, activation='relu'), 
    Dense (units=2, activation='softmax')
])
model.summary()


Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_45 (Dense)            (None, 16)                1584      
                                                                 
 dense_46 (Dense)            (None, 32)                544       
                                                                 
 dense_47 (Dense)            (None, 2)                 66        
                                                                 
Total params: 2,194
Trainable params: 2,194
Non-trainable params: 0
_________________________________________________________________


In [140]:
model.compile(optimizer=Adam(learning_rate=0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [141]:
model.fit(x=X_np, y=y_np,
          batch_size=10, epochs=10, shuffle=True, verbose=2)

# looks like label have to be column vector...

Epoch 1/10
12/12 - 1s - loss: 0.6333 - accuracy: 0.6786 - 1s/epoch - 99ms/step
Epoch 2/10
12/12 - 0s - loss: 0.4938 - accuracy: 0.7321 - 31ms/epoch - 3ms/step
Epoch 3/10
12/12 - 0s - loss: 0.3806 - accuracy: 0.8036 - 36ms/epoch - 3ms/step
Epoch 4/10
12/12 - 0s - loss: 0.2929 - accuracy: 0.8839 - 40ms/epoch - 3ms/step
Epoch 5/10
12/12 - 0s - loss: 0.2050 - accuracy: 0.9196 - 32ms/epoch - 3ms/step
Epoch 6/10
12/12 - 0s - loss: 0.1464 - accuracy: 0.9375 - 39ms/epoch - 3ms/step
Epoch 7/10
12/12 - 0s - loss: 0.1025 - accuracy: 0.9464 - 40ms/epoch - 3ms/step
Epoch 8/10
12/12 - 0s - loss: 0.0812 - accuracy: 0.9643 - 40ms/epoch - 3ms/step
Epoch 9/10
12/12 - 0s - loss: 0.0659 - accuracy: 0.9821 - 34ms/epoch - 3ms/step
Epoch 10/10
12/12 - 0s - loss: 0.0579 - accuracy: 0.9821 - 35ms/epoch - 3ms/step
